In [1]:

import os, glob, json, math, warnings
import numpy as np
import pandas as pd

import tensorflow_data_validation as tfdv

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score

from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType, StringTensorType, Int64TensorType
import onnxruntime as ort

warnings.filterwarnings("ignore")

print("Lib versions:")
import sys, tensorflow as tf, sklearn, onnx, tfx
print("Python:", sys.version)
print("TF:", tf.__version__)
print("TFDV:", tfdv.__version__)
print("tfx:", tfx.__version__)
print("sklearn:", sklearn.__version__)
print("pandas:", pd.__version__)
print("onnx:", onnx.__version__)
print("onnxruntime:", ort.__version__)

2025-09-20 13:29:36.072534: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-09-20 13:29:36.075815: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-09-20 13:29:36.143958: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-09-20 13:29:36.403490: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-09-20 13:29:37.258241: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warnin

Lib versions:
Python: 3.10.18 (main, Sep  8 2025, 22:51:44) [GCC 12.2.0]
TF: 2.16.1
TFDV: 1.16.1
tfx: 1.16.0
sklearn: 1.5.1
pandas: 1.5.3
onnx: 1.17.0
onnxruntime: 1.22.1


In [2]:
df = pd.read_parquet("../../data/test_parquet_data.parquet")

In [3]:
df.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0


In [4]:

import os
from datetime import datetime

# Project paths (mounted volumes are recommended in Docker)
PIPELINE_NAME = "poc_ch3_5"
BASE_DIR = os.path.abspath("../../")  # change to a mounted volume e.g., /workspace
DATA_DIR = os.path.join(BASE_DIR, "data")
PIPELINE_OUTPUT_BASE = os.path.join(BASE_DIR, "artifacts")
PIPELINE_ROOT = os.path.join(PIPELINE_OUTPUT_BASE, PIPELINE_NAME)
METADATA_PATH = os.path.join(PIPELINE_ROOT, "metadata", "metadata.db")
SERVING_MODEL_DIR = os.path.join(PIPELINE_ROOT, "serving_model")

# for p in [DATA_DIR, PIPELINE_ROOT, SERVING_MODEL_DIR]:
#     os.makedirs(p, exist_ok=True)

print("BASE_DIR:", BASE_DIR)
print("DATA_DIR:", DATA_DIR)
print("PIPELINE_ROOT:", PIPELINE_ROOT)
print("METADATA_PATH:", METADATA_PATH)


BASE_DIR: /
DATA_DIR: /data
PIPELINE_ROOT: /artifacts/poc_ch3_5
METADATA_PATH: /artifacts/poc_ch3_5/metadata/metadata.db
